# Carregamento dos dados

In [1]:
import pandas as pd
data = pd.read_csv('apps_reviews_validacao.xlsx - apps_reviews.csv', sep=',')
data.describe()

,reviewId,app_name,content,sentiment_polarity,sentiment,Unnamed: 5
count,3011,3011,3011,3011,3011,1
unique,3011,10,3011,3,7,1
top,01e23c15-44bf-46b7-b280-40e880d9d49b,Shopee,O app da Shopee é razoável. Apesar de ser fáci...,negative,disgust,X
freq,1,303,1,1742,952,1


In [2]:
data.loc[0,'content']

'O app da Shopee é razoável. Apesar de ser fácil de utilizar, apresenta alguns bugs. Por exemplo: Sugestão de palavra errada no campo de pesquisa que acarreta na busca errada. Sugestões de produtos que não têm nada a ver com coisas de nosso interesse. Dificuldade de acesso de alguns jogos ou travamento durante alguma jogada. Falhas para carregar certos anexos para avaliação dos produtos. No geral, são problemas que se pode contornar, pois não ocorrem com tanta frequência.'

# Pré-processamento

In [3]:
import regex # trabalhar com expressões regulares
data['content'] = data['content'].apply(lambda x: regex.sub(r'[^\p{Latin}]', u' ', str(x)))
data.loc[0,'content']

'O app da Shopee é razoável  Apesar de ser fácil de utilizar  apresenta alguns bugs  Por exemplo  Sugestão de palavra errada no campo de pesquisa que acarreta na busca errada  Sugestões de produtos que não têm nada a ver com coisas de nosso interesse  Dificuldade de acesso de alguns jogos ou travamento durante alguma jogada  Falhas para carregar certos anexos para avaliação dos produtos  No geral  são problemas que se pode contornar  pois não ocorrem com tanta frequência '

# Colocando textos minusculo

In [4]:
data['content'] = data['content'].apply(lambda x: str(x).lower())
data.loc[0,'content']

'o app da shopee é razoável  apesar de ser fácil de utilizar  apresenta alguns bugs  por exemplo  sugestão de palavra errada no campo de pesquisa que acarreta na busca errada  sugestões de produtos que não têm nada a ver com coisas de nosso interesse  dificuldade de acesso de alguns jogos ou travamento durante alguma jogada  falhas para carregar certos anexos para avaliação dos produtos  no geral  são problemas que se pode contornar  pois não ocorrem com tanta frequência '

# Removendo stopwords

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Baixar os recursos necessários
nltk.download('stopwords')
nltk.download('punkt')

def remove_stopwords(texto):
    # Obter a lista de stopwords em português
    stops_list = stopwords.words("portuguese")
    # Tokenizar o texto
    word_tokens = word_tokenize(texto)
    # Remover as stopwords
    texto_sem_stops = [w for w in word_tokens if w.lower() not in stops_list]
    return " ".join(texto_sem_stops)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
data['content'] = data['content'].apply(remove_stopwords)
data.loc[0,'content']

'app shopee razoável apesar fácil utilizar apresenta alguns bugs exemplo sugestão palavra errada campo pesquisa acarreta busca errada sugestões produtos têm nada ver coisas interesse dificuldade acesso alguns jogos travamento durante alguma jogada falhas carregar certos anexos avaliação produtos geral problemas pode contornar pois ocorrem tanta frequência'

# Carregamento dados

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import numpy as np


# Preenchendo valores nulos na coluna 'content'
data['content'] = data['content'].fillna('')

# Filtrando o DataFrame para remover linhas onde 'sentiment' é uma string vazia
data = data[data['sentiment'] != '']

# Ou, se você quiser remover linhas onde 'sentiment' é nulo
data = data[data['sentiment'].notna()]

# Word2Vec

In [14]:
# Treinamento do modelo Word2Vec
sentences = [sentence.split() for sentence in data['content']]
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Função para obter a média dos vetores das palavras
def get_mean_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

# Aplicar a função para todas as reviews
X = np.array([get_mean_vector(sentence.split(), model) for sentence in data['content']])
y = np.array(data['sentiment'])

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinamento do modelo de classificação
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Função de tokenização
def tokenize(text):
    return text.lower().split()

# Função para prever o sentimento de um review
def predict_sentiment(review):
    tokens = tokenize(review)
    review_vector = get_mean_vector(tokens, model)
    review_vector = review_vector.reshape(1, -1)  # Transformar em formato 2D
    return clf.predict(review_vector)[0]  # Retornar a previsão


In [15]:
# Previsão e avaliação
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

       anger       0.41      0.35      0.38       153
     disgust       0.48      0.61      0.53       188
        fear       0.00      0.00      0.00         5
   happiness       0.40      0.22      0.28        64
     neutral       0.00      0.00      0.00        19
     sadness       0.54      0.62      0.58       172
    surprise       0.00      0.00      0.00         2

    accuracy                           0.48       603
   macro avg       0.26      0.26      0.25       603
weighted avg       0.45      0.48      0.46       603



C:\Users\arthu\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\arthu\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\arthu\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
# Obter categorias únicas de y_test
categorias = np.unique(y_test)

# Ordenar as categorias
labels = sorted(categorias)

# Calcular a matriz de confusão
cm = confusion_matrix(y_test, y_pred)

# Exibir a matriz de confusão usando seaborn
sns.heatmap(cm, square=True, annot=True, cmap='RdBu', cbar=False, 
            xticklabels=labels,
            yticklabels=labels)

plt.xlabel("Classe real")
plt.ylabel("Classe predita")
plt.title("Matriz de Confusão para Sentiment")
plt.show()


# Teste

In [22]:
import pandas as pd

# Lista de avaliações e emoções retiradas do Google Play refêrentes oo app da shopee
dataTeste ={
    'review': ['Compro a bastante tempo, nunca tive problemas, o que mudou nestes últimos dois meses, a encomenda sai, mas não chega, a data de entrega é alterada automaticamente no aplicativo e a plataforma não resolve a citação, não faz o reembolso e nisso, estou a quase dois meses sem receber a encomenda e sem reembolso, a empresa não soluciona nada! E assim se repete com novas compras!',
    'As lojas são boas, tem bons vendedores. Mas a Shopee é péssima. Tem um atendimento ao cliente péssimo, não sabem o que estão fazendo. Logística terrível e nunca entregam no prazo. E não sabem dar uma resposta coerente. Pode falar com 10 atendentes diferentes e todos vão falar uma versão diferente. O mínimo é uma estrela, mas se tivesse a opção zero, essa seria a minha com toda a certeza. Acho que as palavras incompetência e descaso resumem o serviço prestado.',
    'Eu até que ainda compro no App,mas,depois que comprei um produto,onde o vendedor garantiu ser original,porém não era original e o produto estragou em 1 mês e eu não pude pedir a devolução ou reembolso,fiquei muito desanimado com a Shopee. E,além de tudo, fizeram um programa fidelidade que é a maior enganação, me rebaixaram de nível, mesmo tendo alcançado os pontos necessários pra subir de nível. Esse programa é somente pra recompensar a própria shoppe e nunca os usuários.'],
    'emotion': ['anger','anger','sadness']
    }

df = pd.DataFrame(dataTeste)

print(df.head())


                                              review  emotion
0  Compro a bastante tempo, nunca tive problemas,...    anger
1  As lojas são boas, tem bons vendedores. Mas a ...    anger
2  Eu até que ainda compro no App,mas,depois que ...  sadness


In [23]:
df['review'] = df['review'].apply(lambda x: regex.sub(r'[^\p{Latin}]', u' ', str(x)))
df['review'] = df['review'].apply(lambda x: str(x).lower())
df['review'] = df['review'].apply(remove_stopwords)

df.loc[0,'review']

'compro bastante tempo nunca problemas mudou nestes últimos dois meses encomenda sai chega data entrega alterada automaticamente aplicativo plataforma resolve citação faz reembolso nisso quase dois meses receber encomenda reembolso empresa soluciona nada assim repete novas compras'

In [30]:
emotion = df

emotion['previsao'] = emotion['review'].apply(predict_sentiment)

print('Emoção prevista: ')
print(emotion['previsao'])
print()
print('Emoção real: ')
print(emotion['emotion'])

Emoção prevista: 
0    anger
1    anger
2    anger
Name: previsao, dtype: object

Emoção real: 
0      anger
1      anger
2    sadness
Name: emotion, dtype: object
